<a href="https://colab.research.google.com/github/Amoldwalunj/Customer-complaints-text-classification/blob/main/text_classification_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
import numpy as np
    
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras .preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
STOPWORDS= set(stopwords.words('english'))

In [4]:
df=pd.read_csv('https://github.com/Amoldwalunj/Customer-complaints-text-classification/blob/main/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [5]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57448,2/29/2020,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I am attempting to make a payment toward my st...,None,"Nelnet, Inc.",KS,None,None,Consent provided,Web,2/29/2020,Closed with explanation,Yes,NaN,3549178
57449,2/11/2020,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,Received letter for {$480.00}. Original credit...,Company has responded to the consumer and the ...,"The Receivable Management Services LLC, New Yo...",AZ,853XX,None,Consent provided,Web,2/18/2020,Closed with explanation,Yes,NaN,3527928
57450,2/29/2020,Debt collection,Other debt,Communication tactics,"Used obscene, profane, or other abusive language",entire time 10 years until XX/XX/2020. XXXX ma...,Company has responded to the consumer and the ...,"Convergent Resources, Inc.",NJ,8101,None,Consent provided,Web,2/29/2020,Closed with explanation,Yes,NaN,3549238
57451,1/16/2020,Checking or savings account,Checking account,Problem with a lender or other company chargin...,Transaction was not authorized,I am a customer with Wells Fargo Bank. Recentl...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,AZ,852XX,None,Consent provided,Web,1/22/2020,Closed with explanation,Yes,NaN,3498566


In [6]:

STOPWORDS= set(stopwords.words('english'))

df['Consumer complaint narrative']= df['Consumer complaint narrative'].apply(lambda x: ' '.join([word for word in x.split() if word not in (STOPWORDS)]))

In [7]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,"This auto loan opened XX/XX/2020 XXXX, NC BB &...",Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX 2019 I noticed debt {$620.00} credit b...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated Capital One, XXXX XX/XX/XXXX XXXX 20...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. Capital One, letter...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt incurred due medical malpractice ( X...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57448,2/29/2020,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I attempting make payment toward student loans...,None,"Nelnet, Inc.",KS,None,None,Consent provided,Web,2/29/2020,Closed with explanation,Yes,NaN,3549178
57449,2/11/2020,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,Received letter {$480.00}. Original creditor d...,Company has responded to the consumer and the ...,"The Receivable Management Services LLC, New Yo...",AZ,853XX,None,Consent provided,Web,2/18/2020,Closed with explanation,Yes,NaN,3527928
57450,2/29/2020,Debt collection,Other debt,Communication tactics,"Used obscene, profane, or other abusive language",entire time 10 years XX/XX/2020. XXXX makes bl...,Company has responded to the consumer and the ...,"Convergent Resources, Inc.",NJ,8101,None,Consent provided,Web,2/29/2020,Closed with explanation,Yes,NaN,3549238
57451,1/16/2020,Checking or savings account,Checking account,Problem with a lender or other company chargin...,Transaction was not authorized,I customer Wells Fargo Bank. Recently money wi...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,AZ,852XX,None,Consent provided,Web,1/22/2020,Closed with explanation,Yes,NaN,3498566


In [8]:
df['Consumer complaint narrative'][0]

"This auto loan opened XX/XX/2020 XXXX, NC BB & T name. I NEVER North Carolina I NEVER resident. I filed dispute twice credit bureaus times BB & T claimed accurate loan. Which I aware today. I tried contact BB & T multiple times I never gotten live person. I n't drive I never owned car before. I knowledge account I checked XXXXXXXX XXXX noticed it. I've tried twice dispute it. Additionally I never received bills information account. This last resort trying remove fraudulent loan account."

In [50]:
df['Product'].value_counts()

Debtcollection              21772
Creditcardorprepaidcard     13193
Mortgage                     9799
Checkingorsavingsaccount     7003
Studentloan                  2950
Vehicleloanorlease           2736
Name: Product, dtype: int64

In [9]:
df['Product']= df['Product'].replace(' ', '',regex=True)

In [10]:
import torch
!pip install flair
import flair

     |████████████████████████████████| 286kB 7.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 12.7MB/s 
     |████████████████████████████████| 1.2MB 27.8MB/s 
     |████████████████████████████████| 2.2MB 20.8MB/s 
     |████████████████████████████████| 798kB 54.3MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 3.3MB 45.9MB/s 
     |████████████████████████████████| 870kB 46.8MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=b5185dbf89bc61e00aacb739dc63cb3a779d41ade0018f97e65f26698cc422f7
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for langdetect: fil

In [11]:
df1= df[['Consumer complaint narrative','Product']]

In [12]:
#rename columns
df1.columns= ['text','labels']

In [13]:
df1.head()

,text,labels
0,"This auto loan opened XX/XX/2020 XXXX, NC BB &...",Vehicleloanorlease
1,In XXXX 2019 I noticed debt {$620.00} credit b...,Debtcollection
2,"As stated Capital One, XXXX XX/XX/XXXX XXXX 20...",Vehicleloanorlease
3,"Please see CFPB case XXXX. Capital One, letter...",Checkingorsavingsaccount
4,This debt incurred due medical malpractice ( X...,Debtcollection


In [14]:
df_fst=df1.copy()

In [15]:
df_fst.head()

,text,labels
0,"This auto loan opened XX/XX/2020 XXXX, NC BB &...",Vehicleloanorlease
1,In XXXX 2019 I noticed debt {$620.00} credit b...,Debtcollection
2,"As stated Capital One, XXXX XX/XX/XXXX XXXX 20...",Vehicleloanorlease
3,"Please see CFPB case XXXX. Capital One, letter...",Checkingorsavingsaccount
4,This debt incurred due medical malpractice ( X...,Debtcollection


In [16]:
'__label__' + df_fst['labels'].astype(str)

0              __label__Vehicleloanorlease
1                  __label__Debtcollection
2              __label__Vehicleloanorlease
3        __label__Checkingorsavingsaccount
4                  __label__Debtcollection
                       ...                
57448                 __label__Studentloan
57449              __label__Debtcollection
57450              __label__Debtcollection
57451    __label__Checkingorsavingsaccount
57452              __label__Debtcollection
Name: labels, Length: 57453, dtype: object

In [17]:
df_fst['labels']= '__label__' + df_fst['labels'].astype(str)

In [18]:
df_fst= df_fst[['labels', 'text']]

In [19]:
df_fst.head()

,labels,text
0,__label__Vehicleloanorlease,"This auto loan opened XX/XX/2020 XXXX, NC BB &..."
1,__label__Debtcollection,In XXXX 2019 I noticed debt {$620.00} credit b...
2,__label__Vehicleloanorlease,"As stated Capital One, XXXX XX/XX/XXXX XXXX 20..."
3,__label__Checkingorsavingsaccount,"Please see CFPB case XXXX. Capital One, letter..."
4,__label__Debtcollection,This debt incurred due medical malpractice ( X...


In [20]:
#splitting dataset into 3
train,test,dev= np.split(df1, [int(.6*len(df1)),int(0.8*len(df1))])

In [21]:
#create a folder for CSV
!mkdir -p data

In [22]:
train.to_csv("data/train.csv")
test.to_csv("data/test.csv")
dev.to_csv("data/dev.csv")

In [23]:
#splitting fastText format into 3

train_fst,test_fst,dev_fst= np.split(df_fst, [int(.6*len(df_fst)),int(0.8*len(df_fst))])

In [24]:
!mkdir -p data_fst

In [25]:
train_fst.to_csv("data_fst/train.csv", sep='\t',index=False, header=False)
test_fst.to_csv("data_fst/test.csv", sep='\t',index=False, header=False)
dev_fst.to_csv("data_fst/dev.csv", sep='\t',index=False, header=False)

In [26]:
#Building out corpus
#CSVClassificationCorpus
#ClassificationCorpus

In [27]:
from flair.datasets import ClassificationCorpus, CSVClassificationCorpus
from flair.data import Corpus

In [28]:
#for CSV
df1.columns

Index(['text', 'labels'], dtype='object')

In [29]:
#create column mapping

Column_name_map ={2: 'label_topic', 1:'text'}

In [30]:
#location of the CSV
data_folder= 'data/'

In [31]:
#create corpus for CSV

corpus_csv: Corpus= CSVClassificationCorpus(data_folder=data_folder, column_name_map=Column_name_map, skip_header=True, delimiter=',')

2021-04-13 03:17:35,660 Reading data from data
2021-04-13 03:17:35,662 Train: data/train.csv
2021-04-13 03:17:35,663 Dev: data/dev.csv
2021-04-13 03:17:35,664 Test: data/test.csv


In [32]:
#method 2 using FastText Format

data_folder_fst= 'data_fst/'

In [33]:
corpus_fst:Corpus =ClassificationCorpus(data_folder_fst)

2021-04-13 03:17:36,492 Reading data from data_fst
2021-04-13 03:17:36,494 Train: data_fst/train.csv
2021-04-13 03:17:36,496 Dev: data_fst/dev.csv
2021-04-13 03:17:36,498 Test: data_fst/test.csv


In [34]:
#creating label dictionary for CSV

label_dict_csv= corpus_csv.make_label_dictionary()

2021-04-13 03:17:37,380 Computing label dictionary. Progress:


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  data = _utils.pin_memory.pin_memory(data)
100%|██████████| 45962/45962 [03:13<00:00, 237.99it/s]

2021-04-13 03:20:50,685 [b'Vehicleloanorlease', b'Debtcollection', b'Checkingorsavingsaccount', b'Mortgage', b'Creditcardorprepaidcard', b'Studentloan']


In [35]:
#creating label dictionary for Fasttext

label_dict_fst= corpus_fst.make_label_dictionary()

2021-04-13 03:20:50,698 Computing label dictionary. Progress:


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  data = _utils.pin_memory.pin_memory(data)
100%|██████████| 45962/45962 [03:13<00:00, 237.50it/s]

2021-04-13 03:24:04,439 [b'Vehicleloanorlease', b'Debtcollection', b'Checkingorsavingsaccount', b'Mortgage', b'Creditcardorprepaidcard', b'Studentloan']


In [36]:
#Working with the Word Embedding

from flair.embeddings import FlairEmbeddings, WordEmbeddings, StackedEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings

In [37]:
#create Embedding

word_embedding = [FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

2021-04-13 03:24:05,139 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpyy8hhyi5


100%|██████████| 19689779/19689779 [00:01<00:00, 9846227.59B/s]

2021-04-13 03:24:07,628 copying /tmp/tmpyy8hhyi5 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2021-04-13 03:24:07,651 removing temp file /tmp/tmpyy8hhyi5


2021-04-13 03:24:12,466 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp715uspga


100%|██████████| 19689779/19689779 [00:02<00:00, 9489928.76B/s]

2021-04-13 03:24:15,046 copying /tmp/tmp715uspga to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt


2021-04-13 03:24:15,075 removing temp file /tmp/tmp715uspga


In [38]:
document_embedding= DocumentLSTMEmbeddings(word_embedding, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


**Build and train model**

In [39]:
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [40]:
clf =TextClassifier(document_embedding, label_dictionary=label_dict_fst)

In [41]:
#Training
trainer= ModelTrainer(clf, corpus_fst)

In [45]:
trainer.train('data_fst', max_epochs=1)

2021-04-13 05:33:20,958 ----------------------------------------------------------------------------------------------------
2021-04-13 05:33:20,961 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2048, out_features=256, bias=True)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  data = _utils.pin_memory.pin_memory(data)


2021-04-13 05:44:42,013 	0.4754
2021-04-13 05:44:42,015 
Results:
- F-score (micro) 0.4754
- F-score (macro) 0.2182
- Accuracy 0.4754

By class:
                          precision    recall  f1-score   support

      Vehicleloanorlease     0.0000    0.0000    0.0000       540
          Debtcollection     0.4951    0.9548    0.6520      4199
Checkingorsavingsaccount     0.2500    0.0007    0.0014      1426
                Mortgage     0.5596    0.1598    0.2486      2028
 Creditcardorprepaidcard     0.4018    0.4125    0.4071      2737
             Studentloan     0.0000    0.0000    0.0000       561

               micro avg     0.4754    0.4754    0.4754     11491
               macro avg     0.2844    0.2546    0.2182     11491
            weighted avg     0.4064    0.4754    0.3793     11491
             samples avg     0.4754    0.4754    0.4754     11491

2021-04-13 05:44:42,016 ----------------------------------------------------------------------------------------------------


{'dev_loss_history': [],
 'dev_score_history': [],
 'test_score': 0.4754,
 'train_loss_history': []}